In [1]:
import requests
import os
from bs4 import BeautifulSoup
import re

import urllib.parse

In [2]:
siteUrl = "http://carsonline-ads.com/colsite/col"
siteUrl = "?".join([siteUrl, "cmd="])
siteUrl = "&".join([siteUrl, "do_search=Search+Now"])
siteUrl = "&".join([siteUrl, "make_id=0"])
siteUrl = "&".join([siteUrl, "member_id=-1"])
siteUrl = "&".join([siteUrl, "model_id=0"])
siteUrl = "&".join([siteUrl, "section_id=0"])
siteUrl = "&".join([siteUrl, "sortByDesc=year%2Cmake_id%2Cmodel_id%2Cdate_online"])
siteUrl = "&".join([siteUrl, "state_id=-1"])
siteUrl = "&".join([siteUrl, "type_id=0"])
siteUrl = "&".join([siteUrl, "use=UC2_SearchSite"])
print(siteUrl)
seen_urls_array = []

http://carsonline-ads.com/colsite/col?cmd=&do_search=Search+Now&make_id=0&member_id=-1&model_id=0&section_id=0&sortByDesc=year%2Cmake_id%2Cmodel_id%2Cdate_online&state_id=-1&type_id=0&use=UC2_SearchSite


In [ ]:
for i in range(1900, 2017):
    tagPageUrl = "&".join([siteUrl, "year_max=" + str(i+1)])
    tagPageUrl = "&".join([tagPageUrl, "year_min=" + str(i)])
    
    # Retrieve the page with tag results and set it up to be scraped
    tagPage = requests.get(url=tagPageUrl)
    tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

    links = tagPageSoup.find_all("a", class_="TD-POSTING")
    
    for link in links:
        url = urllib.parse.urljoin(siteUrl, re.sub(' ', '%20', link['href']))
        match = re.search(r"cmd=redirect", url)
        if not match:
            if url not in seen_urls_array:
                seen_urls_array.append(url)
                try:
                    page = requests.get(url=url)
                    soup = BeautifulSoup(page.content, 'lxml')
                    model = soup.select("#ctl00_cphBody_inv1_lblYearMakeModel")
                    if(len(model)):
                        model = model[0].get_text().strip()
                    else:
                        model = soup.select('#yearMakeModel')
                        if(len(model)):
                            model = model[0].get_text().strip()
                        else:
                            model = "Unknown"
                    price = soup.select("#vehiclePrice > p:nth-of-type(1)")
                    if(len(price)):
                        price = price[0].get_text().strip()
                    else:
                        price = "Unknown"
                    description = soup.select('#vehicleDescription > p')
                    if(len(description)):
                        description = description[0].get_text().strip()
                    else:
                        description = ""
                    if(len(description)):
                        match = re.search(r"\(?[0-9]{3}\)?[-. ]?[0-9]{3}[-. ]?[0-9]{4}", description)
                        if match:
                            phone = match.group()
                        else:
                            phone = ""
                    if not len(phone):
                        description = soup.select('#vehicleLocation')
                        if(len(description)):
                            description = description[0].get_text().strip()
                        else:
                            description = ""
                        if(len(description)):
                            match = re.search(r"\(?[0-9]{3}\)?[-. ]?[0-9]{3}[-. ]?[0-9]{4}", description)
                            if match:
                                phone = match.group()
                            else:
                                phone = ""
                    if len(phone):
                        bigFile = open('carsonline-ads.txt', 'a')
                        bigFile.write(model + '\t' + price + '\t' + phone + '\n')
                        bigFile.close()
                except:
                    pass
        else:
            seen_urls_array.append(url)